#**Part 2: Feature Engineering**

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [20]:
df_allmedia = pd.read_csv('After Removing Outliers allmedia.csv')
df_air = pd.read_csv('After Removing Outliers_Air.csv')
df_water = pd.read_csv('After Removing Outliers Water.csv')
df_land = pd.read_csv('After Removing Outliers Land.csv')

In [21]:
# Standardize the column names
df_allmedia.rename(columns={"Sum of release to all media (<1tonne)": "Total Release"}, inplace=True)
df_air.rename(columns={"Total Release to Air": "Total Release"}, inplace=True)
df_water.rename(columns={"Total Release to Water": "Total Release"}, inplace=True)
df_land.rename(columns={"Total Release to Land": "Total Release"}, inplace=True)

# Add a new column to identify the dataset source
df_allmedia["Source"] = "allmedia"
df_air["Source"] = "air"
df_water["Source"] = "water"
df_land["Source"] = "land"

In [22]:
# Display the shapes before merging
print("Shape of allmedia dataset:", df_allmedia.shape)
print("Shape of air dataset:", df_air.shape)
print("Shape of water dataset:", df_water.shape)
print("Shape of land dataset:", df_land.shape)

Shape of allmedia dataset: (44262, 13)
Shape of air dataset: (620083, 13)
Shape of water dataset: (50562, 13)
Shape of land dataset: (7523, 13)


##Merging Datasets

In [23]:
# Combine the datasets
combined_df = pd.concat([df_allmedia, df_air, df_water, df_land], ignore_index=True)

#check the new dataset shape
combined_df.shape


(722430, 13)

##Feature Engineering for Target Variable

In [24]:
# Define the relevant categorical columns for grouping
categorical_columns = [
    "NPRI_ID", "Company_Name", "NAICS", "NAICS Title",
    "PROVINCE", "CAS_Number", "Substance Name (English)",
    "Estimation_Method", "Source"
]

# Sort the dataset by the categorical columns and a numeric column for proper ordering
combined_df.sort_values(by=categorical_columns + ['Reporting_Year'], inplace=True)

# Calculate the Rate of Change for 'Total Release' within each group
combined_df['Rate of Change'] = combined_df.groupby(categorical_columns)['Total Release'].pct_change()

# Handle missing values in 'Rate of Change' by setting them to zero
combined_df['Rate of Change'] = combined_df['Rate of Change'].fillna(0)

#double check that there are no missing values
combined_df.isnull().sum()

,0
Reporting_Year,0
NPRI_ID,0
Company_Name,0
NAICS,0
NAICS Title,0
PROVINCE,0
Latitude,0
Longitude,0
CAS_Number,0
Substance Name (English),0


##Feature Engineering for Relative Years

In [25]:
print(combined_df.columns)


Index(['Reporting_Year', 'NPRI_ID', 'Company_Name', 'NAICS', 'NAICS Title',
       'PROVINCE', 'Latitude', 'Longitude', 'CAS_Number',
       'Substance Name (English)', 'Estimation_Method', 'Total Release',
       'Source', 'Rate of Change'],
      dtype='object')


In [27]:
# Assuming combined_df is your dataset
combined_df["Reporting_Year"] = combined_df["Reporting_Year"].astype(int)

# Grouping by the categorical columns
combined_df.sort_values(by=categorical_columns + ["Reporting_Year"], inplace=True)

# Creating the relative year columns
grouped = combined_df.groupby(categorical_columns)
combined_df["Year-1"] = grouped["Total Release"].shift(1).fillna(0)
combined_df["Year-2"] = grouped["Total Release"].shift(2).fillna(0)
combined_df["Year-3"] = grouped["Total Release"].shift(3).fillna(0)
combined_df["Year-4"] = grouped["Total Release"].shift(4).fillna(0)
combined_df["Year-5"] = grouped["Total Release"].shift(5).fillna(0)

# Display the transformed DataFrame
print(combined_df.head())


        Reporting_Year  NPRI_ID                       Company_Name     NAICS  \
57951             2002      1.0  ALBERTA PACIFIC FOREST INDUSTRIES  322112.0   
77573             2003      1.0  ALBERTA PACIFIC FOREST INDUSTRIES  322112.0   
105504            2004      1.0  ALBERTA PACIFIC FOREST INDUSTRIES  322112.0   
134421            2005      1.0  ALBERTA PACIFIC FOREST INDUSTRIES  322112.0   
163392            2006      1.0  ALBERTA PACIFIC FOREST INDUSTRIES  322112.0   

                NAICS Title PROVINCE   Latitude   Longitude  CAS_Number  \
57951   Chemical pulp mills       AB  54.923116 -112.861867  10049-04-4   
77573   Chemical pulp mills       AB  54.923116 -112.861867  10049-04-4   
105504  Chemical pulp mills       AB  54.923116 -112.861867  10049-04-4   
134421  Chemical pulp mills       AB  54.923116 -112.861867  10049-04-4   
163392  Chemical pulp mills       AB  54.923116 -112.861867  10049-04-4   

       Substance Name (English)                     Estimation_Metho

##Encoding Categorical Features

Observation:

NPRI ID is numeric, and the same Company Name repeats for each NPRI ID, but with slight formatting variations.

Same observation with NAICS and NAICS title; CAS_number and Substance Name

Therefore, to lessen the task for encoding categorical, we will use the id: NPRI ID, NAICS, and CAS_number


In [28]:
# List of categorical numerical columns
categorical_numerical = ["NPRI_ID", "NAICS", "CAS_Number"]

# List of categorical non-numerical columns
categorical_non_numerical = ["PROVINCE", "Estimation_Method", "Source"]

# Calculate the number of unique values for each column
unique_counts = combined_df[categorical_non_numerical].nunique()

# Display the unique counts
print(unique_counts)


PROVINCE             13
Estimation_Method    14
Source                4
dtype: int64


In [32]:
from sklearn.preprocessing import LabelEncoder

# Initialize a LabelEncoder
label_encoder = LabelEncoder()

# Apply Label Encoding to the specified columns
for col in categorical_non_numerical:
    combined_df[col] = label_encoder.fit_transform(combined_df[col])

# Display a preview of the dataset
print(combined_df.dtypes)

Reporting_Year                int64
NPRI_ID                     float64
Company_Name                 object
NAICS                       float64
NAICS Title                  object
PROVINCE                      int64
Latitude                    float64
Longitude                   float64
CAS_Number                   object
Substance Name (English)     object
Estimation_Method             int64
Total Release               float64
Source                        int64
Rate of Change              float64
Year-1                      float64
Year-2                      float64
Year-3                      float64
Year-4                      float64
Year-5                      float64
dtype: object


##Normalization for Numerical features

In [33]:
from sklearn.preprocessing import MinMaxScaler

# Select numerical columns for normalization
numerical_columns = ["Total Release", "Rate of Change"]

# Initialize MinMaxScaler
scaler = MinMaxScaler()

# Apply normalization
combined_df[numerical_columns] = scaler.fit_transform(combined_df[numerical_columns])


##Convert dataset to CSV for ML Regression Model

In [34]:
# Create a copy of the DataFrame without the specified columns for extraction
extracted_df = combined_df.drop(columns=['Company_Name', 'NAICS Title', 'Substance Name (English)'])

# Save the modified DataFrame to a new CSV file
extracted_df.to_csv("Regression_dataset.csv", index=False)